In [ ]:
#importing modules
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
from tensorflow import keras
import tensorflow as tf
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Initialising the model architecture
model = keras.Sequential([
    # keras.layers.MaxPooling2D((2,2),input_shape = (28,28,1)),
    # keras.layers.MaxPooling2D((2,2)),
    keras.layers.RandomRotation(0.5,input_shape = (14,14,1)),

    keras.layers.Conv2D(filters = 16, padding = 'same', kernel_size = (3,3), activation = 'relu'),

    # keras.layers.Conv2D(filters = 32, padding = 'same', kernel_size = (3,3), activation = 'relu'),
    # keras.layers.Conv2D(filters = 64, padding = 'same', kernel_size = (3,3), activation = 'relu'),
    # keras.layers.MaxPooling2D((2,2)),
    # keras.layers.Conv2D(filters = 128, padding = 'same', kernel_size = (3,3), activation = 'relu'),
    keras.layers.Conv2D(filters = 256, padding = 'same', kernel_size = (3,3), activation = 'relu'),

    keras.layers.Flatten(),
    keras.layers.Dense(1000, activation = 'relu'),
    # keras.layers.Dense(11, activation = 'relu'),
    # keras.layers.Dropout(0.05),
    keras.layers.Dense(11, activation = 'sigmoid')
])

# Compiling the model
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = [keras.metrics.CategoricalAccuracy()])

# Adding Model Summary
model.summary()

In [ ]:
# Restore the weights
model.load_weights('/content/Tata_Steel_Model')

In [ ]:
# recursive function to remove connected pixels for a line in the patch, similar to the earlier patch function recurse
def recurse2(arr,x,y,dir,poss, lines = []):
  dirs = [[1,0],[-1,0],[0,1],[0,-1],[1,1],[-1,1],[-1,-1],[-1,1]]

  arr[x, y] = 0

  # to get line positions
  lines.append([x,y])
  try:
    dirs.remove(dir)
  except:
    pass

  for i in dirs:
    x_2 = x + i[0]
    y_2 = y + i[1]

    # extra condition to check if pixel in the patch positions provided by the recurse function
    if(x_2>= arr.shape[0] or y_2>= arr.shape[1] or [x_2,y_2] not in poss):
      continue

    # taking dir relative to the new pixel
    dir2 = [-i[0],-i[1]]

    # checking if pixel value is 1 (white)
    if (arr[x_2,y_2] == 1):

      # recursing again with the new pixel value
      arr, _ = recurse2(arr,x_2,y_2,dir2,poss.copy(),lines)

  return arr, lines

In [ ]:
# sliding window cnn function
def slide_cnn(model, arr, k_size, itr_size=None, stride_b=1, stride_h=1):

  # getting the breadth,height
  b = arr.shape[1]
  h = arr.shape[0]

  # setting window = kernel size if it is None
  if itr_size == None:
    itr_size = k_size

  # calculating number of iterations along breadth and height
  h_itr = (b - (itr_size[0] - 1) - 1)//stride_h + 1
  b_itr = (h - (itr_size[1]-1) - 1)//stride_b + 1

  # setting last 0th neuron prediction (indicating absence of a number) as 1, thus number is absent in start
  y_l = 1

  # pos array to store start of presence of numbers
  pos = []

  # iterating the window and sliding it
  for i in range(b_itr):
    for j in range(h_itr):

      # taking the window from the image array
      img = arr[i*stride_h:itr_size[0]+i*stride_h, j*stride_b:itr_size[1]+j*stride_b]

      img2 = cv2.cvtColor(arr, cv2.COLOR_GRAY2BGR)
      img2 = cv2.rectangle(img2, (i*stride_h,j*stride_b), (itr_size[0]+i*stride_h,itr_size[1]+j*stride_b), (0,0,255), 2)

      cv2_imshow(img2)

      # resizing it to the kernel size
      img_r = cv2.resize(img, (k_size[0], k_size[1]), interpolation=cv2.INTER_AREA)

      # reshaping into a 1 elemental 3d array
      img_f = np.zeros((1,img_r.shape[0],img_r.shape[1]))
      img_f[0,:,:] = img_r

      # making predictions from model
      prediction = model.predict(img_f)

      # converting prediction to integers
      y_pred = np.array([np.argmax(i) for i in prediction])

      # getting y_pred_pr or prediction for presence or absence of a number, 0 means present, 1 means absent
      y_pred_pr = np.zeros(y_pred.shape)
      y_pred_pr[y_pred != 0] = 1

      # checking if previous value was 0 (present) and current is 1 (absent), thus indicating end of a continous stretch of numbers (so we got the desired region to remove)
      if y_pred_pr[0] == 1 and y_l == 0:

        # setting default positions as the value where first presence was encountered
        pos1 = pos[0]
        pos2 = pos[1]

        # checking if pos was lower than 0 and setting to 0 if yes
        if pos1 < 0 :
          pos1 = 0

        if pos2 < 0 :
          pos2 = 0

        # setting posses variable to store the positions
        posses = []

        # checking maximum position values to iterate to 18 actually is for the width of a single window, if multiple windows in a row then this difference is added so that outside of that window is lines checked for
        # pos1 - pos[0] and pos2 - pos[1] is basically for the countering of the starting of the window extending beyond the image corner case
        pos1m = i*stride_h - 2
        pos2m = j*stride_b - 2

        # extracting region for checking multiple line instances
        for k in range(18 - (pos1 - pos[0]) + (pos1m - pos1)):
          for l in range(18 - (pos2 - pos[1]) + (pos2m - pos2)):

            # for i values in range of initial 2 and final 2 rows, thus for all columns
            if (k < 2 - (pos1 - pos[0]) or k > 15 - (pos1 - pos[0]) + (pos1m - pos1)):

              # checking for both boundary exceed condition
              if pos1 + k > arr.shape[0] - 1  and pos2 + l > arr.shape[1] - 1 :
                break

              # checking for single boundary exceed conditions
              if pos1 + k > arr.shape[0] - 1 :
                posses.append([arr.shape[0] - 1, pos2 + l])
                continue

              if pos2 + l > arr.shape[1] - 1 :
                posses.append([pos1 + k, arr.shape[1] - 1])
                continue

              posses.append([pos1 + k, pos2 + l])

            # for i values in range between 2 and 11 thus in the limited column variation region
            elif (k >= 2 - (pos1 - pos[0]) and k <= 15 - (pos1 - pos[0]) + (pos1m - pos1)) and (l < 2 - (pos2 - pos[1])  or l > 15 - (pos2 - pos[1]) + (pos2m - pos2)):

              # checking for both boundary exceed condition
              if pos1 + k > arr.shape[0] - 1  and pos2 + l > arr.shape[1] - 1 :
                break

              # checking for single boundary exceed conditions
              if pos1 + k > arr.shape[0] - 1 :
                posses.append([arr.shape[0] - 1, pos2 + l])
                continue

              if pos2 + l > arr.shape[1] - 1 :
                posses.append([pos1 + k, arr.shape[1] - 1])
                continue

              posses.append([pos1 + i, pos2 + j])


        # print("Positions to check : ",posses)
        img = cv2.cvtColor(arr, cv2.COLOR_GRAY2BGR)
        for pos in posses:
          img = cv2.circle(img, pos, 2, (0, 0, 255), -1)

        cv2_imshow(img)

        # variables to store line and its centroid positions
        lines = []
        g_lines = []

        # iterating through positions of the patch
        for pos in posses:

          x = pos[0]
          y = pos[1]

          # checking if pixel value is 1 (white) thus encountering a line
          if arr[x,y] == 1:

            # print(x,y)

            # calling recursive function to remove the connected white pixels of the currently detected line inside the region and to get lines coming to the point
            arr, line = recurse2(arr,x,y,[0,0],poss=posses.copy(), lines=[])
            # plt.imshow(arr, cmap='gray')

            # adding line value to the list
            lines.append(line)

        # keeping variables to get width of the line or the centroid
        l_width = 0
        g_lines = []

        # getting approximate width of the line
        for line in lines:

          # adding line pixels to the variable
          l_width += len(line)
          line = np.array(line)

          # calculating the centroid
          g_lines.append([line[:,0].mean(), line[:,1].mean()])

        # since we have 2 regions (exit and enter) and 2 columns/rows per part (width of outer - inner box is 2), so we divide by 2*2 = 4
        l_width /= 4
        l_width = round(l_width)

        # erasing the current window to full black
        pos_win_end = [14+pos[0]+2, 14+pos[1]+2]

        # setting ending position to 2nd last row
        if pos_win_end[0] > arr.shape[0] - 2:
          pos_win_end[0] = arr.shape[0] - 2

        # setting ending position to 2nd last column
        if pos_win_end[1] > arr.shape[1] - 2:
          pos_win_end[1] = arr.shape[1] - 2

        # making the whole window black (0)
        arr[pos[0]+2:pos_win_end[0], pos[1]+2:pos_win_end[1]] = 0

        plt.imshow(img_r, cmap = 'gray')
        print(i*stride_h,itr_size[0]+i*stride_h, j*stride_b,itr_size[1]+j*stride_b)

        # drawing the new line to replace the old one which got erased
        cv2.line(arr, round(g_lines[0]), round(g_lines[1]), (1,1,1), l_width)/255

        # setting pos array to empty again
        pos = []

      # recording pos if encountered a new instance of a number
      if y_pred_pr[0] == 0 and y_l == 1:
        pos = [i*stride_h - 2, j*stride_b - 2]

      # recording the last predicted value (for the next iteration)
      y_l = y_pred_pr[0]

In [ ]:
slide_cnn(model, img_cropped_inv_2, k_size=(14,14), itr_size=None)